In [1]:
import pandas as pd
import numpy as np
import re
import os
from IPython.display import HTML

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.decomposition import PCA

from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras import optimizers

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import words
from nltk.corpus import wordnet 
allEnglishWords = words.words() + [w for w in wordnet.words()]
allEnglishWords = np.unique([x.lower() for x in allEnglishWords])

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
stemmer = LancasterStemmer()
import datetime
#import time
import sys
import gc
# libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
np.random.seed(32)


from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Conv1D, MaxPooling1D, Dropout
from keras.utils.np_utils import to_categorical
import keras.models 
from keras.models import model_from_json
from keras import optimizers 

from time import time 
from tensorflow.python.keras.callbacks import TensorBoard

Using TensorFlow backend.


## Data Import

In [2]:
path = "aclImdb/"
positiveFiles = [x for x in os.listdir(path+"train/pos/") if x.endswith(".txt")]
negativeFiles = [x for x in os.listdir(path+"train/neg/") if x.endswith(".txt")]
testFiles1 = [x for x in os.listdir(path+"test/pos/") if x.endswith(".txt")]
testFiles2 = [x for x in os.listdir(path+"test/neg/") if x.endswith(".txt")]

In [3]:
len(testFiles2)

12500

In [4]:
positiveReviews, negativeReviews, pos_testReviews, neg_testReviews = [], [], [], []
for pfile in positiveFiles:
    with open(path+"train/pos/"+pfile, encoding="latin1") as f:
        positiveReviews.append(f.read())
for nfile in negativeFiles:
    with open(path+"train/neg/"+nfile, encoding="latin1") as f:
        negativeReviews.append(f.read())
for tfile in testFiles1:
    with open(path+"test/pos/"+tfile, encoding="latin1") as f:
        pos_testReviews.append(f.read())
for tfile in testFiles2:
    with open(path+"test/neg/"+tfile,encoding="latin1") as f:
        neg_testReviews.append(f.read())
        

In [5]:
reviews_train = pd.concat([
    pd.DataFrame({"review":positiveReviews, "label":1, "file":positiveFiles}),
    pd.DataFrame({"review":negativeReviews, "label":0, "file":negativeFiles})
], ignore_index=True).sample(frac=1, random_state=1)


#test set 
reviews_test= pd.concat([
     pd.DataFrame({"review":pos_testReviews, "label":1, "file":testFiles1}),
    pd.DataFrame({"review":neg_testReviews, "label":0, "file":testFiles2})
], ignore_index=True).sample(frac=1, random_state=1)


print(reviews_train.head())
print(reviews_test.head())

                                                  review  label         file
21492  I have copy of this on VHS, I think they (The ...      0   6844_1.txt
9488   After several extremely well ratings to the po...      1  7290_10.txt
16933  I still don't know why I forced myself to sit ...      0   2740_1.txt
12604  Mt little sister and I are self-proclaimed hor...      0  10094_1.txt
8222   I have personally seen many Disney movies in m...      1   6150_7.txt
                                                  review  label         file
21492  A movie theater with a bad history of past gru...      0   6844_2.txt
9488   "Here On Earth" is a surprising beautiful roma...      1  7290_10.txt
16933  I just watched Descent. Gawds what an awful mo...      0   2740_3.txt
12604  In a nutshell the movie is about a gang war in...      0  10094_4.txt
8222   Instead of watching the recycled history of "P...      1   6150_7.txt


With everything centralized in 1 dataframe, we now perform train, validation and test set splits.

In [6]:
#reviews = reviews[["review", "label", "file"]].sample(frac=1, random_state=1)
#train = reviews[reviews.label!=-1].sample(frac=0.8, random_state=1)
#valid = reviews[reviews.label!=-1].drop(train.index)
#test = reviews[reviews.label==-1]

In [7]:
#print(train.shape)
#print(valid.shape)
#print(test.shape)

In [8]:
HTML(reviews_train.review.iloc[0])

---

## Data Preprocessing

It can perform the following operations.
* Discard non alpha-numeric characters
* Set everything to lower case
* Stems all words using PorterStemmer, and change the stems back to the most occurring existent word.
* Discard non-Egnlish words (not by default).

In [9]:
class Preprocessor(object):
    ''' Preprocess data for NLP tasks. '''

    def __init__(self, alpha=True, lower=True, stemmer=True, english=False):
        self.alpha = alpha
        self.lower = lower
        self.stemmer = stemmer
        self.english = english
        
        self.uniqueWords = None
        self.uniqueStems = None
        
    def fit(self, texts):
        texts = self._doAlways(texts)

        allwords = pd.DataFrame({"word": np.concatenate(texts.apply(lambda x: x.split()).values)})
        self.uniqueWords = allwords.groupby(["word"]).size().rename("count").reset_index()
        self.uniqueWords = self.uniqueWords[self.uniqueWords["count"]>1]
        if self.stemmer:
            self.uniqueWords["stem"] = self.uniqueWords.word.apply(lambda x: PorterStemmer().stem(x)).values
            self.uniqueWords.sort_values(["stem", "count"], inplace=True, ascending=False)
            self.uniqueStems = self.uniqueWords.groupby("stem").first()
        
        #if self.english: self.words["english"] = np.in1d(self.words["mode"], allEnglishWords)
        print("Fitted.")
            
    def transform(self, texts):
        texts = self._doAlways(texts)
        if self.stemmer:
            allwords = np.concatenate(texts.apply(lambda x: x.split()).values)
            uniqueWords = pd.DataFrame(index=np.unique(allwords))
            uniqueWords["stem"] = pd.Series(uniqueWords.index).apply(lambda x: PorterStemmer().stem(x)).values
            uniqueWords["mode"] = uniqueWords.stem.apply(lambda x: self.uniqueStems.loc[x, "word"] if x in self.uniqueStems.index else "")
            texts = texts.apply(lambda x: " ".join([uniqueWords.loc[y, "mode"] for y in x.split()]))
        #if self.english: texts = self.words.apply(lambda x: " ".join([y for y in x.split() if self.words.loc[y,"english"]]))
        print("Transformed.")
        return(texts)

    def fit_transform(self, texts):
        texts = self._doAlways(texts)
        self.fit(texts)
        texts = self.transform(texts)
        return(texts)
    
    def _doAlways(self, texts):
        # Remove parts between <>'s
        texts = texts.apply(lambda x: re.sub('<.*?>', ' ', x))
        # Keep letters and digits only.
        if self.alpha: texts = texts.apply(lambda x: re.sub('[^a-zA-Z0-9 ]+', ' ', x))
        # Set everything to lower case
        if self.lower: texts = texts.apply(lambda x: x.lower())
        return texts  

In [10]:
reviews_train.head()

,review,label,file
21492,"I have copy of this on VHS, I think they (The ...",0,6844_1.txt
9488,After several extremely well ratings to the po...,1,7290_10.txt
16933,I still don't know why I forced myself to sit ...,0,2740_1.txt
12604,Mt little sister and I are self-proclaimed hor...,0,10094_1.txt
8222,I have personally seen many Disney movies in m...,1,6150_7.txt


In [11]:
preprocess = Preprocessor(alpha=True, lower=True, stemmer=True)

In [12]:
%%time
trainX = preprocess.fit_transform(reviews_train.review)
testX =preprocess.fit_transform(reviews_test.review)

Fitted.
Transformed.
Fitted.
Transformed.
Wall time: 3min 20s


In [13]:
trainX.head()

21492    i have copy of this on vhs i think they the te...
9488     after several extremely well rating to the poi...
16933    i still don t know why i forced myself to sit ...
12604    mt little sister and i are self proclaimed hor...
8222     i have person seen many disney movie in my lif...
Name: review, dtype: object

In [14]:
print(preprocess.uniqueWords.shape)
preprocess.uniqueWords[preprocess.uniqueWords.word.str.contains("disappoint")]

(46433, 3)


,word,count,stem
18412,disappointingly,24,disappointingli
18410,disappointed,900,disappoint
18411,disappointing,414,disappoint
18413,disappointment,372,disappoint
18409,disappoint,94,disappoint
18414,disappointments,31,disappoint
18415,disappoints,20,disappoint


In [15]:
print(preprocess.uniqueStems.shape)
preprocess.uniqueStems[preprocess.uniqueStems.word.str.contains("disappoint")]

(30714, 2)


,word,count
stem,,
disappoint,disappointed,900
disappointingli,disappointingly,24


---

## Feature Engineering
Next, we take the preprocessed texts as input and calculate their TF-IDF's ([info](http://www.tfidf.com)). We retain 10000 features per text.

In [16]:
stop_words = text.ENGLISH_STOP_WORDS.union(["thats","weve","dont","lets","youre","im","thi","ha",
    "wa","st","ask","want","like","thank","know","susan","ryan","say","got","ought","ive","theyre"])
tfidf = TfidfVectorizer(min_df=2, max_features=10000, stop_words=stop_words) #, ngram_range=(1,3)

In [17]:
%%time
trainX = tfidf.fit_transform(trainX).toarray()
testX = tfidf.fit_transform(testX).toarray()

Wall time: 15 s


In [18]:
print(trainX.shape)

(25000, 10000)


In [19]:
trainY = reviews_train.label
testY =reviews_test.label

In [20]:
print(trainX.shape, trainY.shape)

print(testX.shape, testY.shape)

(25000, 10000) (25000,)
(25000, 10000) (25000,)


---

## Feature Selection
Next, we take the 10k dimensional tfidf's as input, and keep the 2000 dimensions that correlate the most with our sentiment target. The corresponding words - see below - make sense.

In [21]:
from scipy.stats.stats import pearsonr

In [22]:
getCorrelation = np.vectorize(lambda x: pearsonr(trainX[:,x], trainY)[0])
correlations = getCorrelation(np.arange(trainX.shape[1]))
print(correlations)

[-0.01274173 -0.0180222   0.00906162 ...  0.01701741  0.02091131
  0.00975959]


In [23]:
allIndeces = np.argsort(-correlations)
bestIndeces = allIndeces[np.concatenate([np.arange(1000), np.arange(-1000, 0)])]

In [24]:
vocabulary = np.array(tfidf.get_feature_names())
print(vocabulary[bestIndeces][:10])
print(vocabulary[bestIndeces][-10:])

['grateful' 'loudly' 'evidenced' 'bend' 'bbc' 'perpetrator' 'fascism'
 'endeavors' 'amber' 'perplexed']
['pope' 'stubborn' 'hoskins' 'words' 'teller' 'bond' 'avery' 'waning'
 'work' 'babysitter']


In [25]:
trainX = trainX[:,bestIndeces]
testX  = testX [:,bestIndeces]

In [26]:
print(trainX.shape, trainY.shape)
print(testX.shape,testY.shape)

(25000, 2000) (25000,)
(25000, 2000) (25000,)


---

## Model Architecture
We choose a very simple dense network with 6 layers, performing binary classification.

In [27]:
from tensorflow.python.keras.callbacks import TensorBoard

In [28]:
from time import time 

In [29]:
DROPOUT = 0.5
ACTIVATION = "tanh"

model = Sequential([    
    Dense(int(trainX.shape[1]/2), activation=ACTIVATION, input_dim=trainX.shape[1]),
    Dropout(DROPOUT),
    Dense(int(trainX.shape[1]/2), activation=ACTIVATION, input_dim=trainX.shape[1]),
    Dropout(DROPOUT),
    Dense(int(trainX.shape[1]/4), activation=ACTIVATION),
    Dropout(DROPOUT),
    Dense(100, activation=ACTIVATION),
    Dropout(DROPOUT),
    Dense(20, activation=ACTIVATION),
    Dropout(DROPOUT),
    Dense(5, activation=ACTIVATION),
    Dropout(DROPOUT),
    Dense(1, activation='sigmoid'),
])

Instructions for updating:
Colocations handled automatically by placer.


In [30]:
model.compile(optimizer=optimizers.Adam(0.00005), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               500500    
_________________________________________________________________
dropout_3 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)              

---

## ANN Model Training
Let's go.

In [31]:
EPOCHS = 10
BATCHSIZE = 1500

In [32]:
#submit log reports to tensorflow by specifying a path 
# SAVE MODEL FOR 5 EPOCHS
mc = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5',save_weights_only=True, period=5)
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.fit(trainX, trainY, epochs=EPOCHS, batch_size=BATCHSIZE, validation_split=0.2,verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 12s 580us/step - loss: 0.7008 - accuracy: 0.5121 - val_loss: 0.6780 - val_accuracy: 0.7222
Epoch 2/10
20000/20000 [==============================] - 9s 474us/step - loss: 0.6859 - accuracy: 0.5438 - val_loss: 0.6604 - val_accuracy: 0.7814
Epoch 3/10
20000/20000 [==============================] - 11s 550us/step - loss: 0.6665 - accuracy: 0.5848 - val_loss: 0.6385 - val_accuracy: 0.8144
Epoch 4/10
20000/20000 [==============================] - 10s 489us/step - loss: 0.6439 - accuracy: 0.6296 - val_loss: 0.6089 - val_accuracy: 0.8284
Epoch 5/10
20000/20000 [==============================] - 10s 508us/step - loss: 0.6171 - accuracy: 0.6621 - val_loss: 0.5699 - val_accuracy: 0.8370
Epoch 6/10
20000/20000 [==============================] - 9s 444us/step - loss: 0.5834 - accuracy: 0.7145 - val_loss: 0.5247 - val_accuracy: 0.8418
Epoch 7/10


In [33]:
x = np.arange(EPOCHS)
history = model.history.history

data = [
    go.Scatter(x=x, y=history["acc"], name="Train Accuracy", marker=dict(size=5), yaxis='y2'),
    go.Scatter(x=x, y=history["val_acc"], name="Valid Accuracy", marker=dict(size=5), yaxis='y2'),
    go.Scatter(x=x, y=history["loss"], name="Train Loss", marker=dict(size=5)),
    go.Scatter(x=x, y=history["val_loss"], name="Valid Loss", marker=dict(size=5))
]
layout = go.Layout(
    title="Model Training Evolution", font=dict(family='Palatino'), xaxis=dict(title='Epoch', dtick=1),
    yaxis1=dict(title="Loss", domain=[0, 0.45]), yaxis2=dict(title="Accuracy", domain=[0.55, 1]),
)
py.iplot(go.Figure(data=data, layout=layout), show_link=False)

KeyError: 'acc'

---

## Model Evaluation

### Accuracy & Loss
Let's first centralize the probabilities and predictions with the original train and validation dataframes. Then we can print out the respective accuracies and losses.

In [ ]:
reviews_train["probability"] = model.predict(trainX)
reviews_train["prediction"] = reviews_train.probability-0.5>0
reviews_train["truth"] = reviews_train.label==1
reviews_train.tail()

In [ ]:
print(model.evaluate(trainX, trainY))
print((reviews_train.truth==reviews_train.prediction).mean())

In [ ]:
len(testX)

In [ ]:
reviews_test["probability"] = model.predict(testX)
reviews_test["prediction"] = reviews_test.probability-0.5>0
reviews_test["truth"] = reviews_test.label==1
reviews_test.tail()

In [ ]:
print(model.evaluate(testX, testY))
print((reviews_test.truth==reviews_test.prediction).mean())

### Error Analysis
Error analysis gives us great insight in the way the model is making its errors. Often, it shows data quality issues.

In [ ]:
trainCross = reviews_train.groupby(["prediction", "truth"]).size().unstack()
trainCross

In [ ]:
validCross = reviews_test.groupby(["prediction", "truth"]).size().unstack()
validCross

In [ ]:
truepositives = reviews_test[(reviews_test.truth==True)&(reviews_test.truth==reviews_test.prediction)]
print(len(truepositives), "true positives.")
truepositives.sort_values("probability", ascending=False).head(3)

In [ ]:
truenegatives = reviews_test[(reviews_test.truth==False)&(reviews_test.truth==reviews_test.prediction)]
print(len(truenegatives), "true negatives.")
truenegatives.sort_values("probability", ascending=True).head(3)

In [ ]:
falsepositives = reviews_test[(reviews_test.truth==True)&(reviews_test.truth!=reviews_test.prediction)]
print(len(falsepositives), "false positives.")
falsepositives.sort_values("probability", ascending=True).head(3)

In [ ]:
falsenegatives = reviews_test[(reviews_test.truth==False)&(reviews_test.truth!=reviews_test.prediction)]
print(len(falsenegatives), "false negatives.")
falsenegatives.sort_values("probability", ascending=False).head(3)

This is the review that got predicted as positive most certainly - while being labeled as negative. However, we can easily recognize it as a poorly labeled sample.

In [ ]:
HTML(reviews_test.loc[22148].review)

---

## Model Application

### Custom Reviews
To use this model, we would store the model, along with the preprocessing vectorizers, and run the unseen texts through following pipeline.

In [ ]:
unseen = pd.Series("this movie very good")

In [ ]:
unseen = preprocess.transform(unseen)       # Text preprocessing
unseen = tfidf.transform(unseen).toarray()  # Feature engineering
unseen = unseen[:,bestIndeces]              # Feature selection
probability = model.predict(unseen)[0,0]  # Network feedforward

In [ ]:
print(probability)
print("Positive!") if probability > 0.5 else print("Negative!")